# Capstone Project

In [1]:
import numpy as np
import pandas as pd
import requests # library to handle requests
import random # library for random number generation
import folium # plotting library
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

In [12]:
cred_df = pd.read_csv(r'C:\Courses\ddd\foursquare.csv')
C_ID = cred_df.iloc[0,0]
C_Secret = cred_df.iloc[1,0]

In [13]:
CLIENT_ID = C_ID # your Foursquare ID
CLIENT_SECRET = C_Secret # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

In [14]:
address = 'Hampton Ct Way, Molesey, East Molesey KT8 9AU'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

51.3908872 -0.3451091


In [18]:
search_query = 'Pub'
radius = 1000
print(search_query + ' .... OK!')

Pub .... OK!


In [19]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

In [20]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eedd9b8ed78b8001bcc04eb'},
 'response': {'venues': [{'id': '4c51f989250dd13ae051187e',
    'name': 'The Greyhound Pub',
    'location': {'address': 'Weston Green',
     'lat': 51.382161441161315,
     'lng': -0.3474608057922063,
     'labeledLatLngs': [{'label': 'display',
       'lat': 51.382161441161315,
       'lng': -0.3474608057922063}],
     'distance': 984,
     'postalCode': 'KT7 0JP',
     'cc': 'GB',
     'city': 'Thames Ditton',
     'state': 'Surrey',
     'country': 'United Kingdom',
     'formattedAddress': ['Weston Green',
      'Thames Ditton',
      'Surrey',
      'KT7 0JP',
      'United Kingdom']},
    'categories': [{'id': '4bf58dd8d48988d11b941735',
      'name': 'Pub',
      'pluralName': 'Pubs',
      'shortName': 'Pub',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1592646286',
    'hasPerk': False},
   {'id': '4b9cd9

In [21]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

C:\Users\andre\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,4c51f989250dd13ae051187e,The Greyhound Pub,"[{'id': '4bf58dd8d48988d11b941735', 'name': 'P...",v-1592646286,False,Weston Green,51.382161,-0.347461,"[{'label': 'display', 'lat': 51.38216144116131...",984,KT7 0JP,GB,Thames Ditton,Surrey,United Kingdom,"[Weston Green, Thames Ditton, Surrey, KT7 0JP,..."
1,4b9cd9aef964a520a77e36e3,The Angel,"[{'id': '4bf58dd8d48988d11b941735', 'name': 'P...",v-1592646286,False,Angel Rd.,51.386666,-0.329541,"[{'label': 'display', 'lat': 51.38666603903796...",1179,NaN,GB,Thames Ditton,Surrey,United Kingdom,"[Angel Rd., Thames Ditton, Surrey, United King..."


In [22]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,The Greyhound Pub,Pub,Weston Green,51.382161,-0.347461,"[{'label': 'display', 'lat': 51.38216144116131...",984,KT7 0JP,GB,Thames Ditton,Surrey,United Kingdom,"[Weston Green, Thames Ditton, Surrey, KT7 0JP,...",4c51f989250dd13ae051187e
1,The Angel,Pub,Angel Rd.,51.386666,-0.329541,"[{'label': 'display', 'lat': 51.38666603903796...",1179,NaN,GB,Thames Ditton,Surrey,United Kingdom,"[Angel Rd., Thames Ditton, Surrey, United King...",4b9cd9aef964a520a77e36e3


In [23]:
dataframe_filtered.name

0    The Greyhound Pub
1            The Angel
Name: name, dtype: object

In [24]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [25]:
venue_id = '4c51f989250dd13ae051187e' # ID of The Greyhound Pub
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

In [26]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4c51f989250dd13ae051187e',
 'name': 'The Greyhound Pub',
 'contact': {},
 'location': {'address': 'Weston Green',
  'lat': 51.382161441161315,
  'lng': -0.3474608057922063,
  'labeledLatLngs': [{'label': 'display',
    'lat': 51.382161441161315,
    'lng': -0.3474608057922063}],
  'postalCode': 'KT7 0JP',
  'cc': 'GB',
  'city': 'Thames Ditton',
  'state': 'Surrey',
  'country': 'United Kingdom',
  'formattedAddress': ['Weston Green',
   'Thames Ditton',
   'Surrey',
   'KT7 0JP',
   'United Kingdom']},
 'canonicalUrl': 'https://foursquare.com/v/the-greyhound-pub/4c51f989250dd13ae051187e',
 'categories': [{'id': '4bf58dd8d48988d11b941735',
   'name': 'Pub',
   'pluralName': 'Pubs',
   'shortName': 'Pub',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 5},
 'hasMenu': True,
 'likes': {'count': 12,
  'groups': [{'type': 'others', 'count': 12, 'items': []}],
  'su

In [27]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.4


In [28]:
venue_id = '4b9cd9aef964a520a77e36e3' # ID of The Angel
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.3


In [29]:
result['response']['venue']['tips']['count']

7

In [30]:
#Tips for The Angel
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eeddec3c94979001b1facb9'},
 'response': {'tips': {'count': 7,
   'items': [{'id': '51a874db498eb6d7dcc606b3',
     'createdAt': 1369994459,
     'text': 'Great little pub and a fantastic comedy night the last Thursday of every month!',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/51a874db498eb6d7dcc606b3',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'agreeCount': 2,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'user': {'id': '36005657',
      'firstName': 'Matthew',
      'lastName': 'P',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/L500G2RM0Q4QYHTN.jpg'}},
     'authorInteractionType': 'liked'}]}}}

In [31]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

In [43]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

C:\Users\andre\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.
C:\Users\andre\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.id
0,Great little pub and a fantastic comedy night the last Thursday of every month!,2,0,51a874db498eb6d7dcc606b3,Matthew,P,36005657


In [49]:
user_id = '36005657' # user ID with most agree counts and complete profile

url = 'https://api.foursquare.com/v2/users/{}?client_id={}&client_secret={}&v={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION) # define URL

# send GET request
results = requests.get(url).json()
user_data = results['response']['user']

# display features associated with user
user_data.keys()

KeyError: 'user'

In [ ]:
user_data['tips']

In [ ]:
# define tips URL
url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

# send GET request and get user's tips
results = requests.get(url).json()
tips = results['response']['tips']['items']

# format column width
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips)

# filter columns
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display user's tips
tips_filtered

In [ ]:
tip_id = '5ab5575d73fe2516ad8f363b' # tip id

# define URL
url = 'http://api.foursquare.com/v2/tips/{}?client_id={}&client_secret={}&v={}'.format(tip_id, CLIENT_ID, CLIENT_SECRET, VERSION)

# send GET Request and examine results
result = requests.get(url).json()
print(result['response']['tip']['venue']['name'])
print(result['response']['tip']['venue']['location'])

In [ ]:
user_friends = json_normalize(user_data['friends']['groups'][0]['items'])
user_friends

In [ ]:
user_data

In [ ]:
# 1. grab prefix of photo
# 2. grab suffix of photo
# 3. concatenate them using the image size  
Image(url='https://igx.4sqi.net/img/user/300x300/484542633_mK2Yum7T_7Tn9fWpndidJsmw2Hof_6T5vJBKCHPLMK5OL-U5ZiJGj51iwBstcpDLYa3Zvhvis.jpg')

# Explore a location

In [51]:
address = 'Lephalale, 0555, South Africa'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-23.683333 27.733333


In [52]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)

In [53]:
import requests

In [55]:
results = requests.get(url).json()
'There are {} popular spots around Lephalale.'.format(len(results['response']['groups'][0]['items']))

'There are 4 popular spots around Lephalale.'

In [56]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '5cc03d4cb6eedb002c8f21c4',
  'name': 'Pick n Pay Family Lephalale Square',
  'location': {'address': 'Wells St',
   'lat': -23.67828,
   'lng': 27.73792,
   'labeledLatLngs': [{'label': 'display', 'lat': -23.67828, 'lng': 27.73792}],
   'distance': 731,
   'postalCode': '0555',
   'cc': 'ZA',
   'city': 'Lephalale',
   'state': 'Limpopo',
   'country': 'iNingizimu Afrika',
   'formattedAddress': ['Wells St', 'Ellisras', '0555', 'iNingizimu Afrika']},
  'categories': [{'id': '4d954b0ea243a5684a65b473',
    'name': 'Convenience Store',
    'pluralName': 'Convenience Stores',
    'shortName': 'Convenience Store',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/conveniencestore_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []},
  'venuePage': {'id': '537108502'}},
 'refer

In [57]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

C:\Users\andre\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Pick n Pay Family Lephalale Square,Convenience Store,Wells St,-23.678280,27.737920,"[{'label': 'display', 'lat': -23.67828, 'lng': 27.73792}]",731,0555,ZA,Lephalale,Limpopo,iNingizimu Afrika,"[Wells St, Ellisras, 0555, iNingizimu Afrika]",5cc03d4cb6eedb002c8f21c4
1,Europcar Ellisras,Rental Service,35J J Louis Botha Drive,-23.676566,27.736301,"[{'label': 'display', 'lat': -23.67656574, 'lng': 27.73630073}]",811,0555,ZA,Lephalale,Limpopo,iNingizimu Afrika,"[35J J Louis Botha Drive, Ellisras, 0555, iNingizimu Afrika]",5dd4756119f6a90008fe3ec0
2,Palm Park Hotel,Hotel,NaN,-23.679313,27.740564,"[{'label': 'display', 'lat': -23.679312955127333, 'lng': 27.740564377695506}]",862,NaN,ZA,Lephalale,Limpopo,iNingizimu Afrika,"[Lephalale, iNingizimu Afrika]",4dff6da6ae605b47b175894d
3,Galito's Lephalale Ellisras,Fast Food Restaurant,J Louis Botha Drive,-23.676306,27.736913,"[{'label': 'display', 'lat': -23.67630632, 'lng': 27.73691254}]",863,0555,ZA,Lephalale,Limpopo,iNingizimu Afrika,"[J Louis Botha Drive, Lephalale, 0555, iNingizimu Afrika]",4ff59375e4b0d615836da973


In [58]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

# Explore Trending Venues

In [69]:
address = '7th Street in Johannesburg, South Africa'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-26.1768696 28.008634239105547


In [70]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eede4a9e826ac00214ec820'},
 'response': {'venues': []}}

In [71]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [72]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

In [73]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.features.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [74]:
# display map
venues_map

'Cannot generate visual as no trending venues are available at the moment!'